# 1. Mapa de Casos Encontrados

## Importações

In [8]:
import pandas as pd
import pydeck as pdk
import numpy as np

### Lê os ``.csv``

DataSUS

In [13]:
cols = [
    'DT_SIN_PRI', 'CS_SEXO', 'DT_NASC', 'SG_UF', 'ID_MN_RESI', 'CLASSI_FIN',
    'EVOLUCAO', 'DT_EVOLUCA', 
]
    
df = pd.read_csv("../data/data_sus/INFLUD22-26-06-2025.csv", sep=";", usecols=cols)

df

,DT_SIN_PRI,CS_SEXO,DT_NASC,SG_UF,ID_MN_RESI,CLASSI_FIN,EVOLUCAO,DT_EVOLUCA
0,2022-03-16,M,1938-04-28,RJ,SAO GONCALO,4.0,2.0,2022-03-16
1,2022-06-04,F,1942-07-14,SP,SAO CAETANO DO SUL,5.0,2.0,2022-06-05
2,2022-07-15,F,1928-05-19,SP,SAO CAETANO DO SUL,5.0,2.0,2022-07-20
3,2022-02-20,M,1949-01-02,CE,JUAZEIRO DO NORTE,4.0,1.0,2022-02-21
4,2022-01-08,F,1940-01-15,CE,JUAZEIRO DO NORTE,4.0,1.0,2022-01-15
...,...,...,...,...,...,...,...,...
560572,2022-05-15,M,1957-05-07,MG,CAMPO BELO,4.0,1.0,2025-05-23
560573,2022-07-18,M,2020-10-19,MG,CONTAGEM,2.0,1.0,2022-08-10
560574,2022-04-09,M,2017-02-08,MG,BELO HORIZONTE,4.0,1.0,2025-04-13
560575,2022-05-29,F,1948-04-04,SE,LAGARTO,NaN,NaN,NaN


## Nomeia a classificação final

In [14]:
def get_class_final(id):
    # 1. Usar pd.isna() é a forma mais segura de checar por valores ausentes (NaN, None, etc.)
    if pd.isna(id):
        return "Não Informado"
    
    classifications = {
        1: "SRAG por influenza",
        2: "SRAG por outro vírus respiratório",
        3: "SRAG por outro agente etiológico",
        4: "SRAG não especificado",
        5: "SRAG por covid-19"
    }
    
    # 2. Converte o id para inteiro ANTES de procurar no dicionário
    #    Usar .get() também é mais seguro, pois retorna um valor padrão se a chave não for encontrada.
    return classifications.get(int(id), "ID Inválido")

# A aplicação no DataFrame continua a mesma
df['CLASSI_FIN'] = df['CLASSI_FIN'].apply(get_class_final)

df

,DT_SIN_PRI,CS_SEXO,DT_NASC,SG_UF,ID_MN_RESI,CLASSI_FIN,EVOLUCAO,DT_EVOLUCA
0,2022-03-16,M,1938-04-28,RJ,SAO GONCALO,SRAG não especificado,2.0,2022-03-16
1,2022-06-04,F,1942-07-14,SP,SAO CAETANO DO SUL,SRAG por covid-19,2.0,2022-06-05
2,2022-07-15,F,1928-05-19,SP,SAO CAETANO DO SUL,SRAG por covid-19,2.0,2022-07-20
3,2022-02-20,M,1949-01-02,CE,JUAZEIRO DO NORTE,SRAG não especificado,1.0,2022-02-21
4,2022-01-08,F,1940-01-15,CE,JUAZEIRO DO NORTE,SRAG não especificado,1.0,2022-01-15
...,...,...,...,...,...,...,...,...
560572,2022-05-15,M,1957-05-07,MG,CAMPO BELO,SRAG não especificado,1.0,2025-05-23
560573,2022-07-18,M,2020-10-19,MG,CONTAGEM,SRAG por outro vírus respiratório,1.0,2022-08-10
560574,2022-04-09,M,2017-02-08,MG,BELO HORIZONTE,SRAG não especificado,1.0,2025-04-13
560575,2022-05-29,F,1948-04-04,SE,LAGARTO,Não Informado,NaN,NaN


## Nomeia os critérios

In [15]:
def get_evolucao(id):
    # 1. Usar pd.isna() é a forma mais segura de checar por valores ausentes (NaN, None, etc.)
    if pd.isna(id):
        return "Não Informado"
    
    classifications = {
        1:"Cura",
        2:"Óbito",
        3:"Óbito por outras causas",
        9:"Ignorado"
    }
    
    # 2. Converte o id para inteiro ANTES de procurar no dicionário
    #    Usar .get() também é mais seguro, pois retorna um valor padrão se a chave não for encontrada.
    return classifications.get(int(id), "ID Inválido")

# A aplicação no DataFrame continua a mesma
df['EVOLUCAO'] = df['EVOLUCAO'].apply(get_evolucao)

df

,DT_SIN_PRI,CS_SEXO,DT_NASC,SG_UF,ID_MN_RESI,CLASSI_FIN,EVOLUCAO,DT_EVOLUCA
0,2022-03-16,M,1938-04-28,RJ,SAO GONCALO,SRAG não especificado,Óbito,2022-03-16
1,2022-06-04,F,1942-07-14,SP,SAO CAETANO DO SUL,SRAG por covid-19,Óbito,2022-06-05
2,2022-07-15,F,1928-05-19,SP,SAO CAETANO DO SUL,SRAG por covid-19,Óbito,2022-07-20
3,2022-02-20,M,1949-01-02,CE,JUAZEIRO DO NORTE,SRAG não especificado,Cura,2022-02-21
4,2022-01-08,F,1940-01-15,CE,JUAZEIRO DO NORTE,SRAG não especificado,Cura,2022-01-15
...,...,...,...,...,...,...,...,...
560572,2022-05-15,M,1957-05-07,MG,CAMPO BELO,SRAG não especificado,Cura,2025-05-23
560573,2022-07-18,M,2020-10-19,MG,CONTAGEM,SRAG por outro vírus respiratório,Cura,2022-08-10
560574,2022-04-09,M,2017-02-08,MG,BELO HORIZONTE,SRAG não especificado,Cura,2025-04-13
560575,2022-05-29,F,1948-04-04,SE,LAGARTO,Não Informado,Não Informado,NaN
